# Yandex Cup 2023 RecSys

Одной из ключевых прикладных задач сервиса Яндекс Музыка является автоматическое распознавание жанров музыки. Автоматическая разметка треков по жанрам позволяет более точно подбирать рекомендации с помощью такой фичи, как "Моя волна", адаптирующейся под настроение и музыкальные вкусы пользователей. Так, например, любителям heavy metal не стоит предлагать послушать поп-музыку, а любителям классики - рэп. 

Так как количество музыкальных треков в сервисе Яндекс Музыка достаточно велико, то ручная разметка каждого трека малоэффективна, и возникает потребность в разработке модели, выполняющей разметку треков для рекомендательной системы "Моя волна" автоматически. Более того, не для всех треков имеется хорошая разметка, и хотелось бы знать не только верхнеуровневый жанр (например, рок), так и микрожанр (например, heavy metal).

Таким образом, **целью проекта** является разработка multi-labeled классификатора треков на основе их эмбеддингов, полученных с помощью автоэнкодера.

## Импорт библиотек

В ходе работы были использованы следующие библиотеки:
- ***Pandas*** - для чтения csv файлов, содержащих разметку треков по жанрам для обучающих данных и номера треков из тестовой выборки, для которых необходимо выполнить предсказание жанра.
- ***Numpy*** - для чтения файлов .npy, содержащих эмбеддинги треков
- ***PyTorch*** - фреймворк для обучения нейронных сетей.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import AveragePrecision
from sklearn.model_selection import train_test_split
import os
import gc

In [ ]:
# Настраиваем torch и запишем основные константы
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
NUM_TAGS = 256 
PATH = '/kaggle/input/yandex-cup-2023-recsys/'

## Загрузка данных.

Данные для обучения и тестовые треки хранятся в файлах train.csv и test.csv, а также в каталоге track_embeddings. Загрузим их и создадим загрузчики для их подачи на вход модели.

In [ ]:
df_train = pd.read_csv(PATH + 'train.csv')
df_test = pd.read_csv(PATH + 'test.csv')
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=777)

In [ ]:
track_idx2embeds = {}
for fn in tqdm(os.listdir(PATH + 'track_embeddings')):
    track_idx = int(fn[:-4])
    embeds = torch.from_numpy(np.load(PATH + 'track_embeddings/' + fn))
    track_idx2embeds[track_idx] = embeds

In [ ]:
class TaggingDataset(Dataset):
    def __init__(self, df, testing=False):
        self.df = df
        self.testing = testing
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        track_idx = row.track
        embeds = track_idx2embeds[track_idx]
        if self.testing:
            return track_idx, embeds
        tags = [int(x) for x in row.tags.split(',')]
        target = torch.zeros(NUM_TAGS)
        target[tags] = 1
        return track_idx, embeds, target

In [ ]:
train_dataset = TaggingDataset(df_train)
val_dataset = TaggingDataset(df_val)
test_dataset = TaggingDataset(df_test, True)

In [ ]:
def collate_fn(b):
    track_idxs = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeds = nn.utils.rnn.pad_sequence([x[1] for x in b], batch_first=True)
    targets = np.vstack([x[2] for x in b])
    targets = torch.from_numpy(targets)
    embeds = embeds + torch.empty(embeds.shape).normal_(mean=0.00, std=0.05)
    return track_idxs, embeds, targets

def collate_fn_val(b):
    track_idxs = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeds = nn.utils.rnn.pad_sequence([x[1] for x in b], batch_first=True)
    targets = np.vstack([x[2] for x in b])
    targets = torch.from_numpy(targets)
    return track_idxs, embeds, targets

def collate_fn_test(b):
    track_idxs = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeds = nn.utils.rnn.pad_sequence([x[1] for x in b], batch_first=True)
    return track_idxs, embeds

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_val)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_test)

## Обучение модели

Для предсказания жанра треков была использована нейронная сеть, сочетающая кодировщик трансформера и блоки сверточных слоев. Выводы кодировщика поступают на вход четырех одномерных сверточных блоков с различным размером окна для улавливания паттернов, характерных для того или иного жанра. Выводы сверточных блоков классифицируются при помощи многослойного перцептрона.

In [ ]:
class Network(nn.Module):
    def __init__(
        self,
        num_classes = NUM_TAGS
    ):
        super().__init__() 
        
        # Блок кодировщика
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=768, 
                                                        nhead=4,
                                                        activation='relu',
                                                        batch_first=True,
                                                        dim_feedforward=2048
                                                       )
        self.encoder = nn.TransformerEncoder(self.encoder_layer,
                                             num_layers=1
                                            )
        
        # Сверточные блоки
        self.conv7 = nn.Sequential(
            nn.Conv1d(768, 768, kernel_size=7, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=5, padding='same'),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv1d(768, 768, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=3, padding='same'),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(768, 768, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv1d(768, 768, kernel_size=3, padding='same'),
            nn.ReLU()
        )
        self.conv1 = nn.Sequential(
            nn.Conv1d(768, 768, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        # Многослойный перцептрон
        self.lin = nn.Sequential(
            nn.Dropout1d(),
            nn.Linear(3072, 768),
            nn.ReLU(),
            nn.BatchNorm1d(768),
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.BatchNorm1d(768),
            nn.Linear(768, num_classes)
        )
                                  
    def forward(self, embeds):
        x = self.encoder(embeds)
        x = torch.permute(x, (0, 2, 1))
        c1, _ = torch.max(self.conv7(x), dim=2)
        c2, _ = torch.max(self.conv5(x), dim=2)
        c3, _ = torch.max(self.conv3(x), dim=2)
        c4, _ = torch.max(self.conv1(x), dim=2)
        x = torch.cat([c1, c2, c3, c4], dim=1)
        x = self.lin(x)
        return x


In [ ]:
def predict(model, loader):
    model.eval()
    track_idxs = []
    predictions = []
    with torch.no_grad():
        for data in loader:
            track_idx, embeds = data
            embeds = embeds.to(device)
            pred_logits = model(embeds)
            pred_probs = torch.sigmoid(pred_logits)
            predictions.append(pred_probs.cpu().numpy())
            track_idxs.append(track_idx.numpy())
    predictions = np.vstack(predictions)
    track_idxs = np.vstack(track_idxs).ravel()
    return track_idxs, predictions
            

In [ ]:
def evaluate_model(model, val_loader):
    model.eval()
    metric = MultilabelAveragePrecision(NUM_TAGS, average="macro", thresholds=None,)
    with torch.no_grad():
        for data in iter(val_loader):
            track_idx, embeds, target = data
            embeds = embeds.to(device)
            target = target.int()
            target = target.to(device)
            pred_logits = torch.sigmoid(model(embeds))
            metric(pred_logits, target)
    print(f'---Validation AP: {metric.compute()}---')

In [ ]:
def train_epoch(model, loader, val_loader, criterion, optimizer, microbatches=1):
    model.train()
    running_loss = 0
    optimizer.zero_grad()
    for iteration,data in enumerate(loader):
        torch.cuda.empty_cache()
        track_idxs, embeds, target = data
        embeds = embeds.to(device)
        target = target.to(device)
        pred_logits = model(embeds)
        pred_probs = torch.sigmoid(pred_logits)
        ce_loss = criterion(pred_logits, target)
        ce_loss.backward()
        running_loss += ce_loss.item() / microbatches
        if (iteration + 1) % microbatches == 0:
            optimizer.step()
            optimizer.zero_grad()
            if (iteration + 1) % (microbatches*10) == 0:
                print('   {} batch {} loss {}'.format(
                    datetime.now(), iteration+1, running_loss
                ))
            running_loss = 0
    evaluate_model(model, val_loader)
            

In [ ]:
# Чистим RAM от всякого хлама
torch.cuda.empty_cache()
gc.collect()

model = Network()
criterion = nn.BCEWithLogitsLoss()

epochs = 10
model = model.to(device)
criterion = criterion.to(device)
lr = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
for epoch in tqdm(range(epochs)):
    print(f'lr={lr}')
    train_epoch(model, train_dataloader, val_dataloader, criterion, optimizer, 10)
    lr *= 0.97
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
#torch.save(model, 'transformer_cnn.pt')

## Submission

Загрузим предсказания модели на тестовой выборке и выгрузим полученный файл predictions.csv

In [ ]:
track_idx, predictions = predict(model, test_dataloader)
predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in zip(track_idxs, predictions)
])

In [ ]:
predictions_df.to_csv('prediction.csv', index=False)

## Вывод

В результате проекта была разработана модель на основе гибридной архитектуры Transformer + CNN для автоматического определения музыкальных жанров на основе эмбеддингов музыкальных треков. Средняя точность определения жанра (Average Precision) на валидационной выборке составляет 19 %.